<h1>Proyecto Tecnologías de Búsqueda en la Web</h1>
<h3>Integrantes</h3>
<ul><li>Sebastián Aranda</li><li>Felipe Santander</li></ul>

<h2>Librerías</h2>

In [1]:
import gensim
from gensim import corpora
from gensim.corpora import Dictionary, bleicorpus
from gensim.models import ldamodel
from gensim.models import Phrases

from collections import defaultdict

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

import matplotlib.pyplot as plt
import numpy as np

from random import randint
from __future__ import division

import pyLDAvis
import re
import os
import json
import operator

<h2>Creación del Corpus</h2>

<h6>Rutas a Datasets</h6>

In [2]:
roastme_dataset_path = "Corpus_builder_and_pre/Dataset/"
positive_dataset_path = "Corpus_builder_and_pre/Positivos/"
#balance_dataset_path = "Corpus_builder_and_pre/Neutros&Positivos/"

<h6>Construcción del Corpus</h6>

In [17]:
puncts = ".,:;?!()[]{}~+-\"\'#$%&\/*^|"
digits = "0123456789"
#Create bad words list
bad_words = []
with open('badwords_v2.txt','r') as bad_words_file:
    for word in bad_words_file:
        word = word.split('-')[0].strip()
        word = word.replace('\n','').decode('unicode_escape').encode('ascii','ignore')
        if word != '':
            bad_words.append(word)

#Create the corpus
corpus = list()
for name in os.listdir(roastme_dataset_path):
    if name.endswith('.json'):
        with open(roastme_dataset_path+'/'+name) as f:
            op_json = json.loads(f.read())
            try:
                for child in op_json[1]['data']['children']:    
                    #Extract the comment
                    comment_text = child['data']['body'].encode('ascii', 'ignore').replace('\n', ' ')
                    #Delete links
                    comment_text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', comment_text)
                    for sym in puncts:
                        comment_text = comment_text.replace(sym," ")
                    for num in digits:
                        comment_text = comment_text.replace(num," ")
                    
                    tokens_comment = [word for word in comment_text.lower().split()]    
                    corpus.append(tokens_comment)
                        
                    try:
                        for child in child['data']['replies']['data']['children']:
                            comment_text = child['data']['body'].encode('ascii', 'ignore').replace('\n', ' ')
                            comment_text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', comment_text)
                    
                            for sym in puncts:
                                comment_text = comment_text.replace(sym," ")
                            for num in digits:
                                comment_text = comment_text.replace(num," ")
                        
                            tokens_comment = [word for word in comment_text.lower().split()]    
                            corpus.append(tokens_comment)
                                
                    except Exception as e:
                        #print(e)
                        pass
                    
            except Exception as e:
                #print(e)
                pass

roastme_len = len(corpus)
print("Roastme Dataset Len: "+str(roastme_len))

for name in os.listdir(positive_dataset_path):
    if name.endswith('.json'):
        with open(positive_dataset_path+'/'+name) as f:
            op_json = json.loads(f.read())
            try:
                for child in op_json[1]['data']['children']:    
                    #Extract the comment
                    comment_text = child['data']['body'].encode('ascii', 'ignore').replace('\n', ' ')
                    #Delete links
                    comment_text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', comment_text)
                    for sym in puncts:
                        comment_text = comment_text.replace(sym," ")
                    for num in digits:
                        comment_text = comment_text.replace(num," ")
                    
                    tokens_comment = [word for word in comment_text.lower().split()]        
                    corpus.append(tokens_comment)
                        
                    try:
                        for child in child['data']['replies']['data']['children']:
                            comment_text = child['data']['body'].encode('ascii', 'ignore').replace('\n', ' ')
                            comment_text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', comment_text)
                    
                            for sym in puncts:
                                comment_text = comment_text.replace(sym," ")
                            for num in digits:
                                comment_text = comment_text.replace(num," ")
                        
                            tokens_comment = [word for word in comment_text.lower().split()]
                            corpus.append(tokens_comment)
                                
                    except Exception as e:
                        pass
                    
            except Exception as e:
                pass

positive_len = len(corpus)-roastme_len
print("Dataset Size")
print(len(corpus))
#TODO: guardar corpus de texto con pickle

Roastme Dataset Len: 87664
Dataset Size
87996


<h6>Procesamiento del Corpus</h6>

In [18]:
#Remove lone letters
corpus = [[word for word in doc if len(word)>1] for doc in corpus]

#Remove stopwords
corpus = [[word for word in doc if word not in stopwords.words('english')] for doc in corpus]

# Lemmatize all words in documents.
lemmatizer = WordNetLemmatizer()
corpus = [[lemmatizer.lemmatize(word) for word in doc] for doc in corpus]

#Get Trigrams
#bigram = Phrases(corpus)
#trigram = Phrases(bigram[corpus])
#for idx in range(len(corpus)):
#    for token in trigram[corpus[idx]]:
#            if '_' in token:
#                corpus[idx].append(token)

<h6>Transformación del Corpus a espacio de vectores</h6>

In [20]:
#MultiProcessing
#import multiprocessing as mp

#Create dictionary
dictionary = corpora.Dictionary(corpus)
dictionary.filter_extremes(no_below=len(dictionary)*0.001, no_above=0.75)

#Convert documents to vectors
corpus = [dictionary.doc2bow(doc) for doc in corpus]

#Save corpus and dictionary
corpora.BleiCorpus.serialize('tmp/cyberbullying_corpus.lda-c', corpus)
dictionary.save('tmp/cyberbullying_dictionary.dict')

<h1>Características para Clasificación</h1>
<ul>
<li>Porcentaje de pertenencia al tópico generado de LDA en Roastme Dataset</li>
<li>Densidad de Badwords</li>
<li>TODO: Polaridad</li>
</ul>

In [32]:
#Load the dictionary and corpus
if (os.path.exists('tmp/cyberbullying_dictionary.dict') and os.path.exists('tmp/cyberbullying_corpus.lda-c')):
    print('Creating dictionary...')
    %time dictionary = corpora.Dictionary.load('tmp/cyberbullying_dictionary.dict')
    print('Creating corpus...')
    %time corpus = corpora.BleiCorpus('tmp/cyberbullying_corpus.lda-c')
else:
    print("Create the dictionary and corpus first")

Creating dictionary...
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 2.04 ms
Creating corpus...
CPU times: user 32 ms, sys: 16 ms, total: 48 ms
Wall time: 49 ms


<h3>LDA Features</h3>

In [33]:
#Initialize a model
#print('Creating Tfidf model...')
#tfidf = gensim.models.TfidfModel(corpus)
#corpus_tfidf = tfidf[corpus]
#lda_model_tfidf = ldamodel.LdaModel(corpus_tfidf, num_topics=2, id2word=dictionary)

#%time lda_model = ldamodel.LdaModel(corpus, num_topics=10, id2word=dictionary)
#lda_model.save('tmp/cyberbullying_ldaModel.lda')
#lda_model.print_topics(10,10)

#lda_features = []
#for docbow in corpus:
#    lda_features.append(lda_model[docbow])
lda_features = [randint(0,5) for random_number in range(len(corpus))]

#print(lda_features)

<h3>Bad Words Features</h3>

In [34]:
bad_words_ids = [word_id for word_id, word in dictionary.iteritems() if word in bad_words]
bad_words_features = []

for docbow in corpus:
        bad_words_features.append(sum([freq for word_id,freq in docbow if word_id in bad_words_ids]))
#print(bad_words_features)

<h1>Validación Cruzada</h1>

<h3>Etiquetas</h3>
<p>Asignar etiqueta segun corpus</p>

In [35]:
y1 = [1 for label in range(roastme_len)]
y2 = [0 for label in range(roastme_len,len(corpus))]
y = y1 + y2
print(len(y))

87996


In [36]:
from sklearn.cross_validation import train_test_split
X = zip(lda_features,bad_words_features)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .7)

from sklearn.neighbors import KNeighborsClassifier
myClassifier = KNeighborsClassifier()
myClassifier.fit(X_train, y_train)
predictions = myClassifier.predict(X_test)

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, predictions))

0.99618494107
